# 1. Phân tích dữ liệu 

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import StratifiedKFold
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import tensorflow as tf
import albumentations
import pandas as pd
import numpy as np
import shutil
import os

In [ ]:
#load data
df = pd.read_csv('../input/plant-pathology-2021-fgvc8/train.csv', index_col='image')
#nhãn bằng chữ ban đầu của từng mẫu
labels = df.labels.values.copy()
df

In [ ]:

label_unique = df.labels.unique()
print(label_unique)

In [ ]:
df.value_counts()




In [ ]:
#format train data
train_Y = df['labels'].values.copy() 
# nhìn vào dữ liệu ta thấy rằng 1 mẫu dữ liệu có thể có nhiều nhãn, vậy đây là bài toán đa nhãn
df['labels'] = [x.split(' ') for x in df['labels']] 
classes = ['complex', 'frog_eye_leaf_spot', 'powdery_mildew', 'rust', 'scab', 'healthy']
scores = MultiLabelBinarizer(classes=classes).fit_transform(df['labels'].values)
df = pd.DataFrame(columns=classes, data=scores, index=df.index)
# df.to_csv('train.csv')
df.head()

Ta nhận thấy rằng tuy đây là bài toán đa nhãn nhưng nhìn vào thực tế những mẫu đã được gán nhãn heathy rồi thì sẽ không gán được những nhãn khác, vậy ta sẽ xây dựng model cho 5 nhãn:complex, frog_eye_leaf_spot,  powdery_mildew,  rust,  scab. Mẫu dữ liệu không được gán nhãn nào trong 5 nhãn trên sẽ đc gán là healthy


In [ ]:
x = [df[x].sum() for x in classes]
print(x)
y = classes


In [ ]:
plt.style.use('bmh')
plt.xticks(fontsize =12)
plt.yticks(fontsize =12)
plt.xlabel('Category',fontsize =13)
plt.ylabel('Quantity of Image',fontsize = 13)
plt.xticks(rotation=50)
color_list = ['#f8a709', '#FF6565', '#2CC9FB', '#37DE5B', '#FF74E9', '#2C99FE']
plt.bar(y,x, color = color_list,width=0.6)

plt.show()
print("Số mẫu dữ liệu: ", len(df))

Nhìn vào biểu đồ trên, ta thấy rằng có một số lớp khá ít dữ liệu, nên khi chia dữ liệu dễ bị miss nên ta dùng 5-fold cross validation 

# 2. Chia fold 

In [ ]:
from sklearn.model_selection import KFold
kfold = KFold(n_splits=5, shuffle=True, random_state = 42)
fold_ids = np.zeros(len(df))
for i, (train_ids, val_ids) in enumerate(kfold.split(df.index, labels)):
    fold_ids[val_ids] = i

value_counts = lambda x: pd.Series.value_counts(x, normalize=True)
df_five_folds = pd.DataFrame({
    'origin_data': df.apply(value_counts).loc[1],
    'fold_0': df[fold_ids == 0].apply(value_counts).loc[1],
    'fold_1': df[fold_ids == 1].apply(value_counts).loc[1],
    'fold_2': df[fold_ids == 2].apply(value_counts).loc[1],
    'fold_3': df[fold_ids == 3].apply(value_counts).loc[1],
    'fold_4': df[fold_ids == 4].apply(value_counts).loc[1]})

bar = df_five_folds.plot.bar(figsize=[10, 10], colormap='Spectral', rot=50)

folds = pd.DataFrame({
    'image': df.index,
    'fold': fold_ids})

folds.to_csv('folds.csv', index=False)

Khi sử dụng KFold ta thấy phân bố nhãn trong các fold so với phân bố nhãn trong dữ liệu train ban đầu bị lệch nhau. Với StratifiedKFold ta sẽ thu được tỉ lệ này cân bằng hơn khá nhiều. Với phương pháp này thì nó sẽ chỉ shuffle dữ liệu một lần đầu tiên trước khi bắt đầu chia fold và nó sẽ cố gắng chia sao cho tỷ lệ các class trong các fold là tương đồng nhau.

In [ ]:
from sklearn.model_selection import StratifiedKFold
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state = 42)
fold_ids = np.zeros(len(df))
for i, (train_ids, val_ids) in enumerate(kfold.split(df.index, labels)):
    fold_ids[val_ids] = i

value_counts = lambda x: pd.Series.value_counts(x, normalize=True)
df_five_folds = pd.DataFrame({
    'origin_data': df.apply(value_counts).loc[1],
    'fold_0': df[fold_ids == 0].apply(value_counts).loc[1],
    'fold_1': df[fold_ids == 1].apply(value_counts).loc[1],
    'fold_2': df[fold_ids == 2].apply(value_counts).loc[1],
    'fold_3': df[fold_ids == 3].apply(value_counts).loc[1],
    'fold_4': df[fold_ids == 4].apply(value_counts).loc[1]})

bar = df_five_folds.plot.bar(figsize=[10, 10], colormap='Spectral', rot=50)

folds = pd.DataFrame({
    'image': df.index,
    'fold': fold_ids})

folds.to_csv('folds.csv', index=False)

**Khởi tạo một số tham số**

In [ ]:
root = '../input/plant-pathology-2021-fgvc8/train_images'
classes = [
        'complex', 
        'frog_eye_leaf_spot', 
        'powdery_mildew', 
        'rust', 
        'scab',
        'healthy']
strategy = tf.distribute.get_strategy()
batch_size = 16
#kích thước mỗi ảnh   
img_size = 512
#số fold đã chia
folds = 5 
#số file .tfrec trong mỗi fold
subfolds = 16 
    

# 3. Tạo file tfrecords


> **Định nghĩa một số hàm**

In [ ]:
#tuần tự hóa hình ảnh
def _serialize_image(path, transform=None):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [img_size, img_size])
    image = tf.cast(image, tf.uint8)
    
    if transform is not None:
        image = transform(image=image.numpy())['image']
        
    return tf.image.encode_jpeg(image).numpy()

#tuần tự hóa các mẫu dữ liệu
def _serialize_sample(image, image_name, label):
    feature = {
        'image': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image])),
        'image_name': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image_name])),
        'complex': tf.train.Feature(int64_list=tf.train.Int64List(value=[label[0]])),
        'frog_eye_leaf_spot': tf.train.Feature(int64_list=tf.train.Int64List(value=[label[1]])),
        'powdery_mildew': tf.train.Feature(int64_list=tf.train.Int64List(value=[label[2]])),
        'rust': tf.train.Feature(int64_list=tf.train.Int64List(value=[label[3]])),
        'scab': tf.train.Feature(int64_list=tf.train.Int64List(value=[label[4]])),
        'healthy': tf.train.Feature(int64_list=tf.train.Int64List(value=[label[5]]))}
    sample = tf.train.Example(features=tf.train.Features(feature=feature))
    return sample.SerializeToString()

#tuần tự hóa các fold
def serialize_fold(fold, name, transform=None, bar=None):
    samples = []
    
    for image_name, labels in fold.iterrows():
        path = os.path.join(root, image_name)
        image = _serialize_image(path, transform=transform)
        samples.append(_serialize_sample(image, image_name.encode(), labels))
    
    with tf.io.TFRecordWriter(name + '.tfrec') as writer:
        [writer.write(x) for x in samples]
        
    if bar is not None:
        bar.update(1)

In [ ]:
total = folds * subfolds

with tqdm(total=total) as bar:

    for i in range(folds):

        df_fold = df[fold_ids == i]
        
        folder = f'fold_{i}'
        
        try:
            os.mkdir(folder)
        except FileExistsError:
            shutil.rmtree(folder)
            os.mkdir(folder)
        
        for k, subfold in enumerate(np.array_split(df_fold, subfolds)):
            name=os.path.join(folder, '%.2i-%.3i' % (k, len(subfold)))
            serialize_fold(subfold, name=name, bar=bar)
       

run test